# GroupBy : Split-Apply-Combine

A groupby statement can be thought of as creating a subset of each unique value of a column. operation involves some combination of split, apply and combine.  
- **Splitting** the data into groups based on some criteria.
- **Applying** a function to each group independently. In the apply step, we might wish to do one of following
  - *Aggregation*: Compute a summary statistic (sum, mean, size, count or user defined function)  to each group
  - *Transformation*: perform some group-specific computations and return a like-indexed object e.g Standardize data (zscore) within a group, Filling NAs within groups with a value derived from each group.
  - *Filtration*:  discard some groups, according to a group-wise computation that evaluates True or False. Discard data that belongs to groups with only a few members. Filter out data based on the group sum or mean.
  - Some combination of the above
- Combining the results into a data structure.

In [ ]:
import pandas as pd
from pathlib import Path

HERE = Path('__file__').parent.resolve()
DATA = HERE / 'data'
df = pd.read_csv(DATA / 'ds_salaries_v2.csv')
df.head()

## 1.1 Splitting an object into groups
To create a groupBy object, you can do following

In [ ]:
grouped = df.groupby('job_title')

In [ ]:
# Iterate through groups
for name, group in grouped:
    print(name)
    #print(group)

In [ ]:
# GroupBy object attributes. 
grouped.groups               # dict. Keys: computed unique groups, values: axis labels belonging to each group.

In [ ]:
# Select a group
grouped.get_group("AI Developer")

## 1.2 Applying a function on groups

### 1.2.1 Aggregate
In aggregation, some type of reduction is involved. For example, with 'mean', you take multiple values and replace them with a single value.

#### Basic one variable group aggregation

In [ ]:
# Full/Part time, Contractor, Freelancer
df.groupby('employment_type')['salary_in_usd'].mean()

In [ ]:
# SE: Senior, EN: Entry, EX: Executive, MI: Mid/Intermeidate
df.groupby(['job_title', 'experience_level'])['salary_in_usd'].mean()

#### Built-in aggregation methods
Apart from mean, below are the list of build-in methods. See official documentation https://pandas.pydata.org/docs/user_guide/groupby.html#built-in-aggregation-methods. 

| Method | Description |
| --- | --- |
| mean/median/std/sum/prod | Compute the mean/median/standard-deviation/sum/prod of values in each group |
| count | Compute the number of non-NA values in the groups |
| size | Compute the number of values in each group |
| max/min | Compute the maximum/minimum value in each group |
| any/all | Compute whether any/all of the values in the groups are truthy |
| first/last | Get the first/last occuring value in each group |



In [ ]:
df.groupby('employment_type').first()

One can also calculate multiple summary statistics simultaneously with .describe()

In [ ]:
df.groupby('employment_type')['salary_in_usd'].describe()

#### The aggregate() method
Any reduction method that pandas implements can be passed as a string to aggregate().

In [ ]:
# One can also compute several aggregations at the same time
df.groupby(['employment_type'])['salary_in_usd'].aggregate('sum')
df.groupby(['employment_type'])['salary_in_usd'].aggregate(['mean', 'min', 'max'])

#### Aggregation with user-defined functions
Users can also provide their own user defined functions for custom aggregations.

In [ ]:
# Define a custom aggregation function
def custom_aggregation(values):
    # Compute the sum of after tax salaries
    sum_squares = (values - (values*0.40)).sum()
    return sum_squares

df.groupby(['work_year'])['salary_in_usd'].aggregate(custom_aggregation)

### 1.2.2 Transformation
transfer() takes multiple values and returns a one-to-one transformation of those values. It is used to apply a function to each group of values in a DataFrame. By passing the values from our DataFrame into a function, we can perform computations or manipulations specific to each group independently

In [ ]:
data = {
"Name": ["Inaya", "Aizel", "Sara","Hania", "Umar", "Ayleen", "Monal"],
"City": ["SJC", "SC", "SC", "LM", "SJC", "LM", "SC"],
"Age": [9, 1, 2, 3, 5, 6, 9]
}
df = pd.DataFrame(data, index=[10, 11, 12, 13, 14, 15, 16])
df

In [ ]:
df.groupby('City')['Age'].transform(lambda x: x+10)

In [ ]:
df.groupby('City')['Age'].cumsum()

#### Built-in transformation methods

| Method | Description |
| --- | --- |
| bfill() | Backfill NA values within each group |
| cumcount() | Compute the cumulative count within each group |
| cummin/max() | Compute the cumulative min/max within each group |
| cumsum/cumprod | Compute the cumulative sum/prod within each group |
| rank | Compute the rank of each value within each group |

In [ ]:
df.groupby('City')['Age'].cumcount()

#### The transform() method
Similar to the aggregation method, the transform() method can accept string aliases to the built-in transformation methods in the previous section

In [ ]:
df.groupby('City')['Age'].transform('sum')

### 1.2.3 Filteratoin
A filtration is a GroupBy operation that subsets the original grouping object based on a specified condition or criteria. It may either filter out entire groups, part of groups, or both.

In [ ]:
# Select the top/bottom row(s) of each group. head/tail
df.groupby('City')['Age'].head(1)

In [ ]:
# Select the nth row(s) of each group
df.groupby('City')['Age'].nth()

Users can also use transformation along with Boolean indexing to construct complex functions within groups. 

#### The filter method

In [ ]:
# Filter groups with greater than 2 records.
df.groupby('City')['Age'].filter(lambda x: x.count() > 2)